## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)





In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r'C:\Users\megan\OneDrive\Desktop\Analysis Projects\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv')
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mitsamiouli,KM,-11.3847,43.2844,79.72,80,100,12.15,light rain
1,1,Upernavik,GL,72.7868,-56.1549,-0.18,99,32,4.09,scattered clouds
2,2,Port Lincoln,AU,-34.7333,135.8667,61.03,67,76,15.43,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,46.51,76,75,4.61,light rain
4,4,Chokurdakh,RU,70.6333,147.9167,13.59,92,100,5.77,overcast clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mitsamiouli,KM,-11.3847,43.2844,79.72,80,100,12.15,light rain
6,6,Kapaa,US,22.0752,-159.3190,80.58,85,0,14.97,clear sky
12,12,Padang,ID,-0.9492,100.3543,75.25,76,88,2.84,overcast clouds
15,15,Cabedelo,BR,-6.9811,-34.8339,75.87,83,98,10.58,overcast clouds
20,20,Bambous Virieux,MU,-20.3428,57.7575,75.45,60,20,13.80,few clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mitsamiouli,KM,-11.3847,43.2844,79.72,80,100,12.15,light rain
6,6,Kapaa,US,22.0752,-159.3190,80.58,85,0,14.97,clear sky
12,12,Padang,ID,-0.9492,100.3543,75.25,76,88,2.84,overcast clouds
15,15,Cabedelo,BR,-6.9811,-34.8339,75.87,83,98,10.58,overcast clouds
20,20,Bambous Virieux,MU,-20.3428,57.7575,75.45,60,20,13.80,few clouds
...,...,...,...,...,...,...,...,...,...,...
669,669,Celestun,MX,20.8667,-90.4000,79.09,69,0,19.06,clear sky
672,672,Ternate,ID,0.8000,127.4000,82.17,76,98,2.15,overcast clouds
680,680,Amahai,ID,-3.3333,128.9167,84.87,71,100,0.38,overcast clouds
681,681,Sibolga,ID,1.7427,98.7792,80.06,81,100,1.97,light rain


In [60]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mitsamiouli,KM,79.72,light rain,-11.3847,43.2844,
6,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
12,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,
15,Cabedelo,BR,75.87,overcast clouds,-6.9811,-34.8339,
20,Bambous Virieux,MU,75.45,few clouds,-20.3428,57.7575,
25,Bengkulu,ID,78.78,overcast clouds,-3.8004,102.2655,
26,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
27,Georgetown,MY,76.93,scattered clouds,5.4112,100.3354,
28,Butaritari,KI,82.62,few clouds,3.0707,172.7902,
29,Victoria,HK,83.97,clear sky,22.2855,114.1577,


In [61]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [67]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mitsamiouli,KM,79.72,light rain,-11.3847,43.2844,Foyer ADM
6,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,Mercure Padang
15,Cabedelo,BR,75.87,overcast clouds,-6.9811,-34.8339,Hotel Pigalle
20,Bambous Virieux,MU,75.45,few clouds,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
669,Celestun,MX,79.09,clear sky,20.8667,-90.4000,Posada Lilia
672,Ternate,ID,82.17,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
680,Amahai,ID,84.87,overcast clouds,-3.3333,128.9167,HOTEL ISABELA
681,Sibolga,ID,80.06,light rain,1.7427,98.7792,Hotel Syariah CN Darussalam


In [68]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [69]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))